In [111]:
!ls

Ins_CSV_to_SQL.ipynb customers.csv


In [112]:
!rm customers.sqlite customers_cleaned.csv

rm: customers.sqlite: No such file or directory
rm: customers_cleaned.csv: No such file or directory


In [113]:
# Dependencies
import pandas as pd
import numpy as np
import os

In [114]:
# Path of the CSV file
csvfile = "./customers.csv"


In [115]:
# Read CSV file into a pandas DataFrame
# dtype is: the Data type for data or columns.
# object is The most general dtype. Will be assigned to your column 
  # if column has mixed types (numbers and strings).
    
#http://chris.friedline.net/2015-12-15-rutgers/lessons/python2/03-data-types-and-format.html

df = pd.read_csv(csvfile, dtype=object)

In [116]:
# Preview DataFrame
# Note that some rows are missing gender
df.head()

customerid  firstname     lastname  \
0          1       Luís    Gonçalves   
1          2     Leonie       Köhler   
2          3   François     Tremblay   
3          4      Bjørn       Hansen   
4          5  František  Wichterlová   

                                            company  \
0  Embraer - Empresa Brasileira de Aeronáutica S.A.   
1                                               NaN   
2                                               NaN   
3                                               NaN   
4                                  JetBrains s.r.o.   

                           address                 city state         country  \
0  Av. Brigadeiro Faria Lima, 2170  São José dos Campos    SP          Brazil   
1          Theodor-Heuss-Straße 34            Stuttgart   NaN         Germany   
2                1498 rue Bélanger             Montréal    QC          Canada   
3                 Ullevålsveien 14                 Oslo   NaN          Norway   
4                    Klanova 9/506               Prague   NaN  Czech Republic   

  postalcode               phone                 fax  \
0  12227-000  +55 (12) 3923-5555  +55 (12) 3923-5566   
1      70174    +49 0711 2842222                 NaN   
2    H2G 1A7   +1 (514) 721-4711                 NaN   
3       0171     +47 22 44 22 22                 NaN   
4      14700    +420 2 4172 5555    +420 2 4172 5555   

                      email supportrepid  
0      luisg@embraer.com.br            3  
1     leonekohler@surfeu.de            5  
2       ftremblay@gmail.com            3  
3     bjorn.hansen@yahoo.no            4  
4  frantisekw@jetbrains.com            4

In [117]:
# Use `dropna` to drop any rows where there is missing data
# Notice that index has not been reset and contains a gap
df = df.dropna(axis=0)
df.head(7)

customerid  firstname   lastname  \
0           1       Luís  Gonçalves   
9          10    Eduardo    Martins   
10         11  Alexandre      Rocha   
11         12    Roberto    Almeida   
13         14       Mark    Philips   
14         15   Jennifer   Peterson   
15         16      Frank     Harris   

                                             company  \
0   Embraer - Empresa Brasileira de Aeronáutica S.A.   
9                                   Woodstock Discos   
10                              Banco do Brasil S.A.   
11                                            Riotur   
13                                             Telus   
14                                     Rogers Canada   
15                                       Google Inc.   

                            address                 city state country  \
0   Av. Brigadeiro Faria Lima, 2170  São José dos Campos    SP  Brazil   
9         Rua Dr. Falcão Filho, 155            São Paulo    SP  Brazil   
10               Av. Paulista, 2022            São Paulo    SP  Brazil   
11                 Praça Pio X, 119       Rio de Janeiro    RJ  Brazil   
13                   8210 111 ST NW             Edmonton    AB  Canada   
14              700 W Pender Street            Vancouver    BC  Canada   
15        1600 Amphitheatre Parkway        Mountain View    CA     USA   

    postalcode               phone                 fax  \
0    12227-000  +55 (12) 3923-5555  +55 (12) 3923-5566   
9    01007-010  +55 (11) 3033-5446  +55 (11) 3033-4564   
10   01310-200  +55 (11) 3055-3278  +55 (11) 3055-8131   
11   20040-020  +55 (21) 2271-7000  +55 (21) 2271-7070   
13     T6G 2C7   +1 (780) 434-4554   +1 (780) 434-5565   
14     V6C 1G8   +1 (604) 688-2255   +1 (604) 688-8756   
15  94043-1351   +1 (650) 253-0000   +1 (650) 253-0000   

                            email supportrepid  
0            luisg@embraer.com.br            3  
9        eduardo@woodstock.com.br            4  
10               alero@uol.com.br            5  
11  roberto.almeida@riotur.gov.br            3  
13             mphilips12@shaw.ca            5  
14            jenniferp@rogers.ca            3  
15             fharris@google.com            4

In [119]:
# Drop the 'email' column and reset the index
# Note that after resetting the index, the gap is gone
df = df.drop(['company', 'address', 'city', 'country', 'postalcode', 'state', 'phone', 'fax', 'supportrepid'], axis=1).reset_index(drop=True)

In [123]:
df = df.rename(index=str, columns={"customerid": "id", "firstname": "first_name", "lastname": "last_name"})


In [124]:
df.head(7)


id first_name  last_name                          email
0   1       Luís  Gonçalves           luisg@embraer.com.br
1  10    Eduardo    Martins       eduardo@woodstock.com.br
2  11  Alexandre      Rocha               alero@uol.com.br
3  12    Roberto    Almeida  roberto.almeida@riotur.gov.br
4  14       Mark    Philips             mphilips12@shaw.ca
5  15   Jennifer   Peterson            jenniferp@rogers.ca
6  16      Frank     Harris             fharris@google.com

In [125]:
# Save the cleaned data to a file called `customers_cleaned.csv`
new_csv = "customers_cleaned.csv"
df.to_csv(new_csv, index=False)


In [126]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [127]:
# Create an engine to a SQLite database file called `customers.sqlite`
engine = create_engine("sqlite:///customers.sqlite")

In [128]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [129]:
# Use `declarative_base` from SQLAlchemy to connect your class to your sqlite database
Base = declarative_base()

In [130]:
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html

class Customer(Base):
    __tablename__ = 'customers'

    id = Column(Integer, primary_key=True)
    first_name = Column(Text)
    last_name = Column(Text)
    email = Column(Text)
   
    def __repr__(self):
        return f"id={self.id}, name={self.name}"
        
    # More on __repr__: https://stackoverflow.com/questions/1984162/purpose-of-pythons-repr    

In [131]:
# Use `create_all` to create the customers table in the database
Base.metadata.create_all(engine)

In [132]:
# Load the cleaned csv file into a pandas dataframe
new_df = pd.read_csv(new_csv)


In [133]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
data = new_df.to_dict(orient='records')

In [134]:
print(data)

[{'id': 1, 'first_name': 'Luís', 'last_name': 'Gonçalves', 'email': 'luisg@embraer.com.br'}, {'id': 10, 'first_name': 'Eduardo', 'last_name': 'Martins', 'email': 'eduardo@woodstock.com.br'}, {'id': 11, 'first_name': 'Alexandre', 'last_name': 'Rocha', 'email': 'alero@uol.com.br'}, {'id': 12, 'first_name': 'Roberto', 'last_name': 'Almeida', 'email': 'roberto.almeida@riotur.gov.br'}, {'id': 14, 'first_name': 'Mark', 'last_name': 'Philips', 'email': 'mphilips12@shaw.ca'}, {'id': 15, 'first_name': 'Jennifer', 'last_name': 'Peterson', 'email': 'jenniferp@rogers.ca'}, {'id': 16, 'first_name': 'Frank', 'last_name': 'Harris', 'email': 'fharris@google.com'}, {'id': 17, 'first_name': 'Jack', 'last_name': 'Smith', 'email': 'jacksmith@microsoft.com'}, {'id': 19, 'first_name': 'Tim', 'last_name': 'Goyer', 'email': 'tgoyer@apple.com'}]


In [135]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [136]:
# Save the reference to the `customers` table as a variable called `table`
table = sqlalchemy.Table('customers', metadata, autoload=True)

In [ ]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(table.delete())

In [ ]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(table.insert(), data)

In [ ]:
# didn't work
    # Larger example
    # sql = 'insert into customers (id, first_name, last_name, email, gender) values (?, ?,?,?,?)'
    # conn.executemany(sql, rows)

In [ ]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from customers limit 9").fetchall()